# GraphRNNAutomaton

### Imports

In [1]:
import time
from tqdm import tqdm
#import wandb
import random
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.nn import RNN, Linear, Dropout 
from exporter import read_automatas

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda:0


### Data

In [2]:
automata_property = 'minimal'

In [3]:
path = f'./dataset/{automata_property}_property_automatas'
automatas = read_automatas(path)

In [4]:
def get_batch_graphs(dataset, batch_size):
    dataset1, dataset2 = dataset

    batches1 = [dataset1[i:min(len(dataset1), i+batch_size)] for i in range(0, len(dataset1), batch_size)]
    batches2 = [dataset2[i:min(len(dataset2), i+batch_size)] for i in range(0, len(dataset2), batch_size)]
    return batches1, batches2

### Creating Model

In [5]:
class EdgeMLP(nn.Module):
    def __init__(self, m, input_dim):
        super(EdgeMLP, self).__init__()
        self.m = m

        self.l1 = Linear(in_features=input_dim, out_features=512)
        self.l2 = Linear(in_features=512, out_features=256)
        self.l3 = Linear(in_features=256, out_features=512)
        self.l4 = Linear(in_features=512, out_features=2*m+3)
    
        self.dropout = Dropout(p=.3)

    def forward(self, x):
        res = F.sigmoid(self.l1(x))
        res = self.dropout(res)
        res = F.leaky_relu(self.l2(res), negative_slope=.02)
        res = self.dropout(res)
        res = F.leaky_relu(self.l3(res), negative_slope=.02)
        res = self.dropout(res)
        res = F.sigmoid(self.l4(res))

        return res

In [6]:
class NodeRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, dropout=0):
        super(NodeRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim 
        self.rnn = RNN(input_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)

    def forward(self, x, h):
        _, h = self.rnn(x, h)
        return h[-1]
    
    def get_sos(self, batch_size):
        return torch.zeros((batch_size, 1, self.input_dim))
    
    def get_initial_hidden(self, batch_size):
        return torch.zeros((1, batch_size, self.hidden_dim))

In [7]:
class AutomatonRNN(nn.Module):
    def __init__(self, m, hidden_dim):
        super(AutomatonRNN, self).__init__()
        self.m = m
        self.node_rnn = NodeRNN(2*m+3, hidden_dim)
        self.edge_model = EdgeMLP(m, input_dim=hidden_dim)

    def forward(self, x, h):
        hidden = self.node_rnn(x, h)
        return self.edge_model(hidden), hidden
    
    def get_sos(self, n):
        return self.node_rnn.get_sos(n)

    def get_initial_hidden(self, n):
        return self.node_rnn.get_initial_hidden(n)
        

### Obtaining data from a graph

In [8]:
padding_symbol = 0
def add_padding_to_transitions(graphs):
    max_len = max([len(g) for g in graphs])
    new_graphs = []
    for graph in graphs:
        
        len_diff = max_len-len(graph)

        new_graph = []
        padding = np.full(len_diff, padding_symbol)
        for row in graph:
            new_graph.append(np.concatenate((row, padding)))   

        for _ in range(len_diff):
            new_graph.append(np.full(max_len, padding_symbol))

        new_graphs.append(new_graph)

    return np.array(new_graphs)

def add_padding_to_final_states(all_final_states):
    max_len = max([len(fs) for fs in all_final_states])
    padded_final_states = []
    for final_states in all_final_states:
        len_diff = max_len-len(final_states)
        padding = np.full(len_diff, padding_symbol)
        padded_final_states.append(np.concatenate((final_states, padding)))
    return np.array(padded_final_states)

def add_padding_to_graph(transitions, final_states):
    return add_padding_to_transitions(transitions), add_padding_to_final_states(final_states)

In [9]:
def get_target_conns(graphs, node, m):
    batch_size = graphs.shape[0]

    initial_pos = max(0, node - m)
    in_conns = np.array(graphs[:,initial_pos:node,node], dtype=np.float32)
    loop_con = np.array(np.expand_dims(graphs[:,node,node],1), dtype=np.float32)
    out_conns = np.array(graphs[:,node,initial_pos:node], dtype=np.float32)
    
    padding_size = max(0, m - node)
    padding = np.zeros((batch_size,padding_size),dtype=np.float32)
    y_conns = np.concatenate((padding, in_conns, loop_con, out_conns, padding), 1)
    return torch.tensor(y_conns, dtype=torch.float32)

def get_target_is_final(final_nodes, node):
    return torch.tensor(final_nodes[:,node], dtype=torch.float32).unsqueeze(-1)

def get_target_is_end(nodes, node):
    # we sum 0 to transform bools to int
    return torch.tensor((nodes == node) + 0, dtype=torch.float32).unsqueeze(-1)

def get_nodes(graphs):
    return np.array([len(g) for g in graphs])

In [10]:
def unfold_pred(res, m):
    conns = res[:,:2*m+1]
    final_prob = res[:,2*m+1]
    end_prob = res[:,2*m+2]
    return conns, final_prob, end_prob

### Loss function

In [11]:
def compose_loss(y_hat, y):
    conns_hat, final_prob_hat, end_prob_hat = y_hat
    conns, final_prob, end_prob = y
    # Convert to batch and BCE loss for conns
    conns_loss = nn.BCELoss()(conns_hat, conns)
    # BCE loss for final prob
    final_prob_loss = nn.BCELoss()(final_prob_hat, final_prob)
    # BCE loss for end prob
    end_prob_loss = nn.BCELoss()(end_prob_hat, end_prob)

    # Total loss us the sum of all losses
    return conns_loss + final_prob_loss + end_prob_loss

### Training Loop

In [12]:
def train_model(model, optim, dataset, criterion, epochs, batch_size):
    dataset_len = len(dataset[0])
    for epoch in range(epochs):
        start_time = time.time()
        loss_val = 0
        all_transitions, all_final_states = get_batch_graphs(dataset, batch_size)
        for i, transitions in enumerate(tqdm(all_transitions)):
            final_states = all_final_states[i]
            bs = len(final_states)

            iter_loss = 0
            x = model.get_sos(bs)
            h = model.get_initial_hidden(bs)

            nodes = get_nodes(transitions)
            max_node = max(nodes)

            padded_transitions, padded_final_states = add_padding_to_graph(transitions, final_states)
            
            for node in range(max_node):
                optim.zero_grad()

                # Get targets 
                y_conns = get_target_conns(padded_transitions, node, model.m)
                y_final = get_target_is_final(padded_final_states, node)
                y_end = get_target_is_end(nodes, node)
                y = torch.cat((y_conns, y_final, y_end), 1)

                # Run one iteration of the model
                pred, hidden = model(x, h)
                
                # Compute the loss function
                loss = criterion(pred, y)
                loss.backward(retain_graph=True)
                optim.step()

                # Update hidden and x values for next iteration
                h = hidden.reshape(1,bs,-1).detach().requires_grad_()
                x = pred.reshape(bs,1,-1).detach().requires_grad_()

                # Add the loss value
                iter_loss += loss.item()

            loss_val += iter_loss
            #wandb.log({'train_loss':iter_loss})
        if not epoch%0:
            print(f"Epoch {epoch}, duration: {time.time()-start_time}s -- TRAIN: loss {loss_val/dataset_len}")
                
    return model, loss_val/dataset_len

### Wandb

### Creating model and optimizer

In [13]:
m = 20
hidden_dim = 256
automaton_rnn = AutomatonRNN(m, hidden_dim)
criterion = nn.BCELoss(weight=torch.Tensor(np.ones(2*m+3)*100))

optim = torch.optim.Adam(automaton_rnn.parameters(), lr=.002)

### Run training!

In [14]:
automaton_rnn, training_loss = train_model(automaton_rnn, optim, automatas, criterion, epochs=2, batch_size=32)

 50%|█████     | 1/2 [00:56<00:56, 56.17s/it]

Epoch 0, duration: 56.166069984436035s -- TRAIN: loss 13.841176894831657


100%|██████████| 2/2 [01:48<00:00, 54.24s/it]

Epoch 1, duration: 52.305511474609375s -- TRAIN: loss 13.63853474252224


In [15]:
print(f'Final training loss {training_loss}')

Final training loss 13.63853474252224


In [16]:
class Graph:
    def __init__(self):
        self.nodes = {}
        self.final_nodes = set()
        
    def add_node(self, node, conns, is_final):
        self.nodes[node] = set()
        m = (len(conns)-1)//2
        in_conns = conns[max(0, m-node):m]
        loop_p = float(conns[m])
        out_conns = conns[m+1:len(conns)-max(0,m-node)]

        for target, p_in in enumerate(in_conns):
            p_in = float(p_in)
            in_connection = np.random.choice([False, True], p=[1-p_in, p_in])
            if in_connection:
                self.nodes[target].add(node)

            p_out = float(out_conns[target])
            out_connection = np.random.choice([False, True], p=[1-p_out, p_out])
            if out_connection:
                self.nodes[node].add(target)
        
        loop_connection = np.random.choice([False, True], p=[1-loop_p, loop_p])
        if loop_connection:
            self.nodes[node].add(node)

        if is_final:
            self.final_nodes.add(node)

In [17]:
## REVISAR!!
def generate(model, max_nodes, number_of_graphs):
    with torch.no_grad():
        graphs = [Graph() for _ in range(number_of_graphs)]
        x = model.get_sos(number_of_graphs)
        h = model.get_initial_hidden(number_of_graphs)
        end = False
        node = 0
        while not end:
            x, h = model(x, h)
            conns, final_prob, end_prob = unfold_pred(x, model.m)
            final_prob = float(final_prob)
            is_final = np.random.choice([False, True], p=[1-final_prob, final_prob])
            graph.add_node(node, conns, is_final)
            end_prob = float(end_prob)
            end = np.random.choice([False, True], p=[1-end_prob, end_prob])
            node += 1
            x = x.reshape(1,-1)
            h = h.reshape(1,-1)

            if node > max_nodes:
                end = True

        return graphs

def generate_automatas(model, max_nodes, number_graphs):
    return [convert_to_automata(g) for g in generate(model, max_nodes, number_graphs)]

In [18]:
graph = generate(automaton_rnn, 25)

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
print(f'Final Nodes: {graph.final_nodes}')
print(f'Nodes: {graph.nodes}')

Final Nodes: set()
Nodes: {0: set(), 1: set(), 2: {1, 2}, 3: set(), 4: set(), 5: set(), 6: set(), 7: {6}, 8: {3}}


In [ ]:
def convert_to_automata(g, alphabet):
    nodes = len(g.nodes.values)
    transitions = np.fill((nodes, nodes), set())    
    for origin, dests in g.nodes:
        if len(dests) == 0:
            continue

        sampling_dests = dests
        was_emptied = False
        for symbol in alphabet:
            dest = np.random.choice(list(sampling_dests))
            transitions[origin, dest].add(symbol)
            if not was_emptied:
                sampling_dests.remove(dest)
                
            if len(sampling_dests) == 0:
                sampling_dests = dests
                was_emptied = True

    return Automata(transitions, {str(n) for n in g.final_states}, alphabet, 
                initial_state='0', {str(i):i for i in range(nodes)})
            

In [ ]:
from property_validator import validate_property
def get_metrics(property, automatas):
    return [validate_property(property, a) for a in automatas]